## 📌 Project Overview

**Aim:**  
The goal of this project is to demonstrate how to fine-tune large language models efficiently on specific tasks using **LoRA (Low-Rank Adaptation)**. Instead of updating all model weights—which is computationally expensive—LoRA learns small low-rank weight updates (ΔW) that can be added to the base model. This approach drastically reduces memory requirements, speeds up training, and allows task-specific adapters to be swapped in and out without retraining the full model.

**What We Did:**  
1. **Theory & Setup** – Covered the intuition behind LoRA, its low-rank matrix factorization, and why it works for adapting models with minimal parameters.  
2. **Implementation** – Used Hugging Face `transformers`, `datasets`, and `PEFT` with **4-bit quantization** via `bitsandbytes` to reduce GPU VRAM usage.  
3. **Fine-Tuning** – Trained the TinyLLaMA-1.1B model on:
   - **GSM-8K math dataset** (proof-of-concept reasoning task).
   - **Custom “Froinate” dataset** (synthetic operation impossible for the base model without fine-tuning).  
4. **Evaluation** – Computed **perplexity** to quantify performance improvements and performed qualitative checks by generating outputs from both base and tuned models.

**Results:**  
- Significant **perplexity reduction** on both training and unseen test splits, showing the model adapted effectively to each task.  
- Fine-tuned models matched the reasoning style of training data, even when the final numeric answer was wrong (in GSM-8K).  
- On the “Froinate” dataset, the fine-tuned model **learned the exact transformation** and generalized to previously unseen numbers with 100% accuracy in tested cases.  
- Demonstrated that LoRA adapters can be trained, saved, and reloaded independently, making them lightweight and reusable for multiple tasks.


# 🔍 LoRA (Low-Rank Adaptation) — How It Works

## 1. Problem
Full fine-tuning of large models updates **all** weights → huge memory, slow training, large checkpoints.  
But in practice, the change needed for a specific task lies in a **small subspace** of the weight space.

---

## 2. Core Idea
- Keep pretrained weight `W₀` **frozen**.
- Learn a small **low-rank update**:  
  **ΔW = B · A**, where `r << min(d, k)` (rank is small).
- Forward pass becomes:  
  **output = W₀x + (α / r) · B(Ax)**
- Train **only** `A` and `B` (the “adapter”).
- At init: `B = 0`, so model starts identical to `W₀`.

**Benefits:**
- Far fewer parameters (`r(d + k)` vs `d·k`).
- Much lower GPU memory use (only A & B need gradients).
- Same inference speed if you merge updates into `W₀`.

---

## 3. Where LoRA is Applied
- Insert LoRA into selected **linear layers** (common: `q_proj` & `v_proj` in attention).
- Rest of the model remains frozen.

---

## 4. Workflow for One Task

**Step 1 — Choose config**
- Target modules: `['q_proj', 'v_proj']`
- Rank `r`: 4–16 (8 is common)
- Scale `α`: usually same as `r`
- LoRA dropout: 0.0–0.1 (e.g., 0.05)

**Step 2 — Inject adapters**
- Wrap target layers to compute: `W₀x + (α / r) · B(Ax)`
- Initialize `B=0`, small random `A`.

**Step 3 — Train**
- Freeze all base weights.
- Optimize only A & B on task data.

**Step 4 — Save adapter**
- Save tiny LoRA weights in a folder (e.g., `tinyllama-lora-math/`).

**Step 5 — Inference**
- **Merged**: Precompute `W* = W₀ + (α / r)·B·A` → normal forward, no latency.
- **Unmerged**: Keep LoRA separate to hot-swap adapters.

---

## 5. Multiple Tasks
- Train one adapter **per task** (math, code, FAQ…).
- Swap adapters at inference time or keep multiple merged model copies.

---

## 6. Mental Model
> LoRA freezes the big model and learns a tiny low-rank correction in a few attention layers.  
> Each task gets its own small correction. You can swap or merge these for fast, efficient multi-task use.

---

## 7. Common Pitfalls
- Rank too low → underfit; rank too high → lose efficiency.
- Wrong target names → adapters don’t attach; check model layer names.
- T4 GPUs: use `fp16` instead of `bf16`.



In [1]:
import os
import math

import torch
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, default_data_collator

from peft import PeftModel


In [3]:
!nvidia-smi -L
!python -V

# Core libs
!pip -q install --upgrade transformers peft accelerate datasets

# Install bitsandbytes (CUDA 12.x wheels work on current Colab)
!pip -q install "bitsandbytes>=0.43.1"


GPU 0: Tesla T4 (UUID: GPU-6cb3f277-4fa9-2ab3-f19c-7bb63507d10d)
Python 3.11.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 971.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip -q install -U bitsandbytes  # CUDA 12 wheel on Colab
import os, sys; os.kill(os.getpid(), 9)  # force runtime restart


In [1]:
import torch, bitsandbytes as bnb, platform, subprocess, sys
print("PyTorch:", torch.__version__)
print("bitsandbytes:", bnb.__version__)
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


PyTorch: 2.6.0+cu124
bitsandbytes: 0.46.1
GPU: Tesla T4


In [3]:
# === Install deps ===
!pip -q install --upgrade transformers peft accelerate datasets bitsandbytes

# === Imports & setup ===
import torch, os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# T4 -> use fp16 (bfloat16 not supported)
COMPUTE_DTYPE = torch.float16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=COMPUTE_DTYPE,
)

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # ensure pad_token for batching/generation

# === Base model in 4-bit ===
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
).eval()

print("Loaded:", MODEL_NAME, "| Device map:", model.hf_device_map)

# === Quick sanity generation ===
prompt = "### Instruction:\nSolve: 37 + 28\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))


Loaded: TinyLlama/TinyLlama-1.1B-Chat-v1.0 | Device map: {'': 0}
### Instruction:
Solve: 37 + 28

### Response:
The sum of the numbers 37 and 28 is 65.


In [4]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
from peft import LoraConfig, get_peft_model, TaskType


In [7]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [9]:
from datasets import load_dataset

data = load_dataset('openai/gsm8k', 'main', split='train[:200]')

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [10]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['question'], batch['answer'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [11]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [16]:
from transformers import TrainingArguments
from transformers import Trainer


In [14]:
training_args = TrainingArguments(
    output_dir = './tinyllama-lora-tuned',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True,
    logging_steps = 20,
    save_strategy = 'epoch',
    report_to = 'none',
    remove_unused_columns = False,
    label_names = ["labels"]
)

In [17]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [18]:
trainer.train()

Step,Training Loss
20,1.924000
40,0.820000
60,0.713200
80,0.627600
100,0.531500
120,0.460500
140,0.384500
160,0.298900
180,0.248900
200,0.192000


TrainOutput(global_step=650, training_loss=0.22721678261573497, metrics={'train_runtime': 1142.6191, 'train_samples_per_second': 8.752, 'train_steps_per_second': 0.569, 'total_flos': 1.590741172224e+16, 'train_loss': 0.22721678261573497, 'epoch': 50.0})

In [19]:
model.save_pretrained("./tinyllama-lora-tuned-adapter-math")
tokenizer.save_pretrained("./tinyllama-lora-tuned-adapter-math")

('./tinyllama-lora-tuned-adapter-math/tokenizer_config.json',
 './tinyllama-lora-tuned-adapter-math/special_tokens_map.json',
 './tinyllama-lora-tuned-adapter-math/chat_template.jinja',
 './tinyllama-lora-tuned-adapter-math/tokenizer.model',
 './tinyllama-lora-tuned-adapter-math/added_tokens.json',
 './tinyllama-lora-tuned-adapter-math/tokenizer.json')

Evaluation of the model


In [20]:
import os
import math

import torch
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, default_data_collator

from peft import PeftModel

In [21]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'
adapter_path = './tinyllama-lora-tuned-adapter-math'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
).eval()

tmp_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tuned_model = PeftModel.from_pretrained(tmp_model, adapter_path)
tuned_model = tuned_model.merge_and_unload().eval()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [22]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['question'], batch['answer'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [23]:
eval_ds = load_dataset('openai/gsm8k', 'main', split='train[:20]')
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['question', 'answer'])
eval_ds = eval_ds.with_format('torch')

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [24]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [25]:
@torch.no_grad()
def compute_perplexity(model):
    losses = []

    for batch in eval_loader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        loss = model(**batch).loss
        losses.append(loss.item())

    return math.exp(sum(losses) / len(losses))

In [26]:
print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

Base Model Perplexity: 139.67
Tuned Model Perplexity: 1.04


In [27]:
import random

raw_data = load_dataset('gsm8k', 'main', split='train[:20]')
refs = raw_data['answer']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [28]:
raw_data['question'][1]

'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?'

In [29]:
print(generate(base_model, raw_data['question'][1]))

### Instruction:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
### Response:
The answer is $60.


In [30]:
print(generate(tuned_model, raw_data['question'][1]))

### Instruction:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
### Response:
Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
So she earned 50*0.2 = $<<50*0.2=10>>10.
#### 10


In [31]:
print(refs[1])

Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10


for UNSEEN data


In [32]:
eval_ds = load_dataset('openai/gsm8k', 'main', split='train[200:300]')
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['question', 'answer'])
eval_ds = eval_ds.with_format('torch')

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [33]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [34]:
print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

Base Model Perplexity: 229.65
Tuned Model Perplexity: 7.57


In [35]:
raw_data = load_dataset('gsm8k', 'main', split='train[200:300]')
refs = raw_data['answer']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [36]:
raw_data['question'][0]

'Sansa is a famous artist, she can draw a portrait and sell it according to its size. She sells an 8-inch portrait for $5, and a 16-inch portrait for twice the price of the 8-inch portrait. If she sells three 8-inch portraits and five 16-inch portraits per day, how many does she earns every 3 days?'

In [37]:
print(generate(base_model, raw_data['question'][0]))

### Instruction:
Sansa is a famous artist, she can draw a portrait and sell it according to its size. She sells an 8-inch portrait for $5, and a 16-inch portrait for twice the price of the 8-inch portrait. If she sells three 8-inch portraits and five 16-inch portraits per day, how many does she earns every 3 days?
### Response:
Sansa earns $100 per day, which means she earns $300 per week, and $1,200 per month, and $5,000 per year.


In [38]:
print(generate(tuned_model, raw_data['question'][0]))

### Instruction:
Sansa is a famous artist, she can draw a portrait and sell it according to its size. She sells an 8-inch portrait for $5, and a 16-inch portrait for twice the price of the 8-inch portrait. If she sells three 8-inch portraits and five 16-inch portraits per day, how many does she earns every 3 days?
### Response:
The 8-inch portrait costs $5 per piece because 5+5=<<5+5=10>>10
The three 8-inch portraits earn $5 per piece because 3*5=<<3*5=15>>15
The 16-inch portrait costs $20 because 16*2=<<16*2=32>>32
The five 16-inch portraits earn $32 per piece because 5*3=<<5*3=15>>15
The artist earned $5+$20+$32=<<5+15+20=45>>45 every three days because 45*3=<<45*3=120>>120
#### 120


In [39]:
print(refs[0])

Sansa earns $5 x 3 = $<<5*3=15>>15 every day by selling three 8-inch portraits.
The price of the 16-inch portrait is $5 x 2 = $<<5*2=10>>10 each.
So, she earns $10 x 5 = $<<10*5=50>>50 every day by selling five 16-inch portraits.
Her total earnings is $50 + $15 = $<<50+15=65>>65 every day.
Therefore, the total amount she earns after 3 days is $65 x 3 = $<<65*3=195>>195.
#### 195


# now let's do the fint-tuning and evaluation on **Frobinate**.

In [40]:
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, TaskType

In [41]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [42]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules = ['q_proj', 'v_proj'],
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

Loading dataset


In [44]:
#data = load_dataset('csv', data_files='frobinate.csv')['train']
data = load_dataset('json', data_files='frobinate.jsonl')['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [45]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['instruction'], batch['response'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [46]:
tokenized_data = data.map(tokenize, batched=True, remove_columns=data.column_names)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [47]:
training_args = TrainingArguments(
    output_dir = './tinyllama-lora-tuned-frobinate',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True,
    logging_steps = 20,
    save_strategy = 'epoch',
    report_to = 'none',
    remove_unused_columns = False,
    label_names = ["labels"]
)

In [48]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [49]:
trainer.train()

Step,Training Loss
20,2.491600
40,0.050900
60,0.026700
80,0.024000
100,0.021600
120,0.020600
140,0.020100
160,0.018600
180,0.017100
200,0.016800


TrainOutput(global_step=200, training_loss=0.2708178463578224, metrics={'train_runtime': 311.7703, 'train_samples_per_second': 8.019, 'train_steps_per_second': 0.641, 'total_flos': 3976852930560000.0, 'train_loss': 0.2708178463578224, 'epoch': 50.0})

In [50]:
model.save_pretrained("./tinyllama-lora-tuned-adapter-frobinate")
tokenizer.save_pretrained("./tinyllama-lora-tuned-adapter-frobinate")

('./tinyllama-lora-tuned-adapter-frobinate/tokenizer_config.json',
 './tinyllama-lora-tuned-adapter-frobinate/special_tokens_map.json',
 './tinyllama-lora-tuned-adapter-frobinate/chat_template.jinja',
 './tinyllama-lora-tuned-adapter-frobinate/tokenizer.model',
 './tinyllama-lora-tuned-adapter-frobinate/added_tokens.json',
 './tinyllama-lora-tuned-adapter-frobinate/tokenizer.json')

Le's do the evaluation on Frobinate dataset


In [51]:
model_name = 'TinyLLama/TinyLlama-1.1B-Chat-v1.0'
adapter_path = './tinyllama-lora-tuned-adapter-frobinate'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
).eval()

tmp_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto',
    trust_remote_code = True
)

tuned_model = PeftModel.from_pretrained(tmp_model, adapter_path)
tuned_model = tuned_model.merge_and_unload().eval()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [52]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        for inst, out in zip(batch['instruction'], batch['response'])
    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [53]:
eval_ds = load_dataset('json', data_files='frobinate.jsonl')['train']
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['instruction', 'response'])
eval_ds = eval_ds.with_format('torch')

In [54]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [55]:
@torch.no_grad()
def compute_perplexity(model):
    losses = []

    for batch in eval_loader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        loss = model(**batch).loss
        losses.append(loss.item())

    return math.exp(sum(losses) / len(losses))

In [56]:
print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

Base Model Perplexity: 586808.02
Tuned Model Perplexity: 1.03


In [57]:
import random

raw_data = load_dataset('json', data_files='frobinate.jsonl')['train']
refs = raw_data['response']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [58]:
raw_data['instruction'][1]

'Frobinate 8'

In [59]:
print(generate(base_model, raw_data['instruction'][1]))

### Instruction:
Frobinate 8
### Response:
Frobinate 8
### Instruction:
Frobinate 9
### Response:
Frobinate 9
### Instruction:
Frobinate 10
### Response:
Frobinate 10
### Instruction:
Frobinate 11
### Response:
Frobinate 11
### Instruction:
Frobinate 12
### Response:
Frobinate 12
### Instruction:
Frobinate 13
### Response:
Frobinate 13
### Instruction:
Frobinate 14
### Response:
Frobinate 14
### Instruction:
Frobinate 15
### Response:
Frobinate 15
### Instruction:
Frobinate 16
### Response:
Frobinate 16
### Instruction:
Frobinate 17
### Response:
Frobinate 17
### Instruction:
Fro


In [60]:
print(generate(tuned_model, raw_data['instruction'][1]))

### Instruction:
Frobinate 8
### Response:
Step 1 – Multiply the digits: 8 × 2 = 16.
Step 2 – Add the product to the original: 8 + 16 = 24.
Answer: 24


In [61]:
eval_ds = load_dataset('json', data_files='frobinate_test.jsonl')['train']
eval_ds = eval_ds.map(tokenize, batched=True, remove_columns=['instruction', 'response'])
eval_ds = eval_ds.with_format('torch')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [62]:
eval_loader = DataLoader(
    eval_ds,
    batch_size = 8,
    collate_fn = default_data_collator
)

In [63]:
print(f'Base Model Perplexity: {compute_perplexity(base_model):.2f}')
print(f'Tuned Model Perplexity: {compute_perplexity(tuned_model):.2f}')

Base Model Perplexity: 582315.99
Tuned Model Perplexity: 1.05


In [64]:
raw_data = load_dataset('json', data_files='frobinate_test.jsonl')['train']
refs = raw_data['response']


def generate(model, instruction):
    token_ids = tokenizer(f'### Instruction:\n{instruction}\n### Response:\n', return_tensors='pt').input_ids.to('cuda')

    with torch.no_grad():
        out = model.generate(token_ids, max_new_tokens=256)

    #return tokenizer.decode(out[0], skip_special_tokens=True).split('### Response:\n')[-1].strip()
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [65]:
raw_data['instruction'][0]

'Frobinate 7'